# Two dataset target analysis of transient absorption data

equareasim3


### Reference

DOI: <TODO>

In [ ]:
# imports
from pyglotaran_extras.io import setup_case_study

from glotaran.io import load_dataset, load_parameters, load_scheme

In [ ]:
# extra import for plotting
import matplotlib.pyplot as plt
from pyglotaran_extras.compat import convert
from pyglotaran_extras.plotting.plot_overview import plot_overview
from pyglotaran_extras.plotting.style import PlotStyle

### Location of data files



In [ ]:
MODEL_PATH = "models/model-staging.yml"
PARAMETERS_PATH = "models/parameters-staging.yml"

results_folder, script_folder = setup_case_study(
    output_folder_name="pyglotaran_examples_results_staging",
)

data_path1 = script_folder.joinpath("data/data1.ascii")
data_path2 = script_folder.joinpath("data/data2.ascii")
model_path = script_folder.joinpath(MODEL_PATH)
parameter_path = script_folder.joinpath(PARAMETERS_PATH)

### Create scheme with parameter autocomplete

In [ ]:
from glotaran.utils.json_schema import create_model_scheme_json_schema

create_model_scheme_json_schema("models/schema.json", PARAMETERS_PATH)

### Loading in data files


In [ ]:
# load the data
experiment_data = {"dataset1": load_dataset(data_path1), "dataset2": load_dataset(data_path2)}
# load the (model) scheme (library)
scheme = load_scheme(model_path, format_name="yml")
# load the parameters
parameters = load_parameters(parameter_path)
# attach the data to the scheme
scheme.load_data(experiment_data)

# print(scheme.validate(parameters=parameters))

### Optimizing the global fit


In [ ]:
# Run the optimizer
result = scheme.optimize(parameters, maximum_number_function_evaluations=116)

print(result)

### Saving results to file


In [ ]:
# Save the results
result.save(results_folder, allow_overwrite=True)

### Inspection of results


In [ ]:
# Inspect the results
res1 = convert(result.data["dataset1"])
res2 = convert(result.data["dataset2"])

print(res1)
print(res2)

### Plotting of results

This requires pyglotaran-extras


In [ ]:
# Plot the results

# %% Set subsequent plots to the glotaran style
plot_style = PlotStyle()
plt.rc("axes", prop_cycle=plot_style.cycler)

# %%
fig1, _ = plot_overview(res1, linlog=True)
fig2, _ = plot_overview(res2, linlog=True)

# %%
figure_output_path1 = results_folder / f"plot_overview_{results_folder.name}_1.pdf"
figure_output_path2 = results_folder / f"plot_overview_{results_folder.name}_2.pdf"
fig1.savefig(str(figure_output_path1), bbox_inches="tight")
fig2.savefig(str(figure_output_path2), bbox_inches="tight")
print(results_folder)

In [ ]:
result.data["dataset1"]

In [ ]:
data_array1 = result.data["dataset1"].species_associated_estimation
data_array2 = result.data["dataset2"].species_associated_estimation

import matplotlib.pyplot as plt

plt.plot(data_array1.coords["spectral"], data_array1.to_numpy() - data_array2.to_numpy())
plt.xlabel("Spectral")
plt.ylabel("Value")
plt.title("Delta between species_associated_estimation")
plt.show()

In [ ]:
from glotaran.io import save_dataset

for data_path in results_folder.rglob("*.nc"):
    save_dataset(
        convert(load_dataset(data_path)), results_folder / data_path.name, allow_overwrite=True
    )
    data_path.unlink()